In [8]:
#!pip install xarray
from dask.distributed import Client, progress, LocalCluster
from rechunker import rechunk
import s3fs
import xarray as xr
import zarr
import dask.array as dsa
import shutil
from dask.diagnostics import ProgressBar

## Start the Dask Cluster

In [2]:
cluster = LocalCluster()
client = Client(cluster)
client

<Client: 'tcp://127.0.0.1:36641' processes=4 threads=8, memory=31.46 GB>

## Load the MUR SST Dataset from AWS PDS S3

In [3]:
s3 = s3fs.S3FileSystem(anon=True, client_kwargs=dict(region_name='us-west-2'))
s3_store = s3fs.S3Map(root='mur-sst/zarr', s3=s3, check=False)
ds_zarr = zarr.open_consolidated(s3_store, mode='r') #, mask_and_scale=False) - Do we wa    nt mask_and_scale here?

In [4]:
%%time
ds = xr.open_zarr(s3_store, consolidated=True)

CPU times: user 11 s, sys: 1.04 s, total: 12 s
Wall time: 3min 43s


In [5]:
ds

<xarray.Dataset>
Dimensions:           (lat: 17999, lon: 36000, time: 6443)
Coordinates:
  * lat               (lat) float32 -89.99 -89.98 -89.97 ... 89.97 89.98 89.99
  * lon               (lon) float32 -179.99 -179.98 -179.97 ... 179.99 180.0
  * time              (time) datetime64[ns] 2002-06-01T09:00:00 ... 2020-01-2...
Data variables:
    analysed_sst      (time, lat, lon) float32 dask.array<chunksize=(5, 1799, 3600), meta=np.ndarray>
    analysis_error    (time, lat, lon) float32 dask.array<chunksize=(5, 1799, 3600), meta=np.ndarray>
    mask              (time, lat, lon) float32 dask.array<chunksize=(5, 1799, 3600), meta=np.ndarray>
    sea_ice_fraction  (time, lat, lon) float32 dask.array<chunksize=(5, 1799, 3600), meta=np.ndarray>
Attributes:
    Conventions:                CF-1.7
    Metadata_Conventions:       Unidata Observation Dataset v1.0
    acknowledgment:             Please acknowledge the use of these data with...
    cdm_data_type:              grid
    comment:                    MUR = "Multi-scale Ultra-high Resolution"
    creator_email:              ghrsst@podaac.jpl.nasa.gov
    creator_name:               JPL MUR SST project
    creator_url:                http://mur.jpl.nasa.gov
    date_created:               20200124T010755Z
    easternmost_longitude:      180.0
    file_quality_level:         3
    gds_version_id:             2.0
    geospatial_lat_resolution:  0.009999999776482582
    geospatial_lat_units:       degrees north
    geospatial_lon_resolution:  0.009999999776482582
    geospatial_lon_units:       degrees east
    history:                    created at nominal 4-day latency; replaced nr...
    id:                         MUR-JPL-L4-GLOB-v04.1
    institution:                Jet Propulsion Laboratory
    keywords:                   Oceans > Ocean Temperature > Sea Surface Temp...
    keywords_vocabulary:        NASA Global Change Master Directory (GCMD) Sc...
    license:                    These data are available free of charge under...
    metadata_link:              http://podaac.jpl.nasa.gov/ws/metadata/datase...
    naming_authority:           org.ghrsst
    netcdf_version_id:          4.1
    northernmost_latitude:      90.0
    platform:                   Terra, Aqua, GCOM-W, MetOp-A, MetOp-B, Buoys/...
    processing_level:           L4
    product_version:            04.1
    project:                    NASA Making Earth Science Data Records for Us...
    publisher_email:            ghrsst-po@nceo.ac.uk
    publisher_name:             GHRSST Project Office
    publisher_url:              http://www.ghrsst.org
    references:                 http://podaac.jpl.nasa.gov/Multi-scale_Ultra-...
    sensor:                     MODIS, AMSR2, AVHRR, in-situ
    source:                     MODIS_T-JPL, MODIS_A-JPL, AMSR2-REMSS, AVHRRM...
    southernmost_latitude:      -90.0
    spatial_resolution:         0.01 degrees
    standard_name_vocabulary:   NetCDF Climate and Forecast (CF) Metadata Con...
    start_time:                 20200116T090000Z
    stop_time:                  20200116T090000Z
    summary:                    A merged, multi-sensor L4 Foundation SST anal...
    time_coverage_end:          20200116T210000Z
    time_coverage_start:        20200115T210000Z
    title:                      Daily MUR SST, Final product
    uuid:                       27665bc0-d5fc-11e1-9b23-0800200c9a66
    westernmost_longitude:      -180.0

In [7]:
ds.chunk({'time':6443, 'lat':50, 'lon':100})

distributed.utils_perf - WARNING - full garbage collections took 60% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 61% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 61% CPU time recently (threshold: 10%)


<xarray.Dataset>
Dimensions:           (lat: 17999, lon: 36000, time: 6443)
Coordinates:
  * lat               (lat) float32 -89.99 -89.98 -89.97 ... 89.97 89.98 89.99
  * lon               (lon) float32 -179.99 -179.98 -179.97 ... 179.99 180.0
  * time              (time) datetime64[ns] 2002-06-01T09:00:00 ... 2020-01-2...
Data variables:
    analysed_sst      (time, lat, lon) float32 dask.array<chunksize=(6443, 50, 100), meta=np.ndarray>
    analysis_error    (time, lat, lon) float32 dask.array<chunksize=(6443, 50, 100), meta=np.ndarray>
    mask              (time, lat, lon) float32 dask.array<chunksize=(6443, 50, 100), meta=np.ndarray>
    sea_ice_fraction  (time, lat, lon) float32 dask.array<chunksize=(6443, 50, 100), meta=np.ndarray>
Attributes:
    Conventions:                CF-1.7
    Metadata_Conventions:       Unidata Observation Dataset v1.0
    acknowledgment:             Please acknowledge the use of these data with...
    cdm_data_type:              grid
    comment:                    MUR = "Multi-scale Ultra-high Resolution"
    creator_email:              ghrsst@podaac.jpl.nasa.gov
    creator_name:               JPL MUR SST project
    creator_url:                http://mur.jpl.nasa.gov
    date_created:               20200124T010755Z
    easternmost_longitude:      180.0
    file_quality_level:         3
    gds_version_id:             2.0
    geospatial_lat_resolution:  0.009999999776482582
    geospatial_lat_units:       degrees north
    geospatial_lon_resolution:  0.009999999776482582
    geospatial_lon_units:       degrees east
    history:                    created at nominal 4-day latency; replaced nr...
    id:                         MUR-JPL-L4-GLOB-v04.1
    institution:                Jet Propulsion Laboratory
    keywords:                   Oceans > Ocean Temperature > Sea Surface Temp...
    keywords_vocabulary:        NASA Global Change Master Directory (GCMD) Sc...
    license:                    These data are available free of charge under...
    metadata_link:              http://podaac.jpl.nasa.gov/ws/metadata/datase...
    naming_authority:           org.ghrsst
    netcdf_version_id:          4.1
    northernmost_latitude:      90.0
    platform:                   Terra, Aqua, GCOM-W, MetOp-A, MetOp-B, Buoys/...
    processing_level:           L4
    product_version:            04.1
    project:                    NASA Making Earth Science Data Records for Us...
    publisher_email:            ghrsst-po@nceo.ac.uk
    publisher_name:             GHRSST Project Office
    publisher_url:              http://www.ghrsst.org
    references:                 http://podaac.jpl.nasa.gov/Multi-scale_Ultra-...
    sensor:                     MODIS, AMSR2, AVHRR, in-situ
    source:                     MODIS_T-JPL, MODIS_A-JPL, AMSR2-REMSS, AVHRRM...
    southernmost_latitude:      -90.0
    spatial_resolution:         0.01 degrees
    standard_name_vocabulary:   NetCDF Climate and Forecast (CF) Metadata Con...
    start_time:                 20200116T090000Z
    stop_time:                  20200116T090000Z
    summary:                    A merged, multi-sensor L4 Foundation SST anal...
    time_coverage_end:          20200116T210000Z
    time_coverage_start:        20200115T210000Z
    title:                      Daily MUR SST, Final product
    uuid:                       27665bc0-d5fc-11e1-9b23-0800200c9a66
    westernmost_longitude:      -180.0

## Toy example

In [ ]:
subset = ds_zarr['analysed_sst'].get_orthogonal_selection((slice(0,100), slice(0,100), slice(0,100)))
baby_zarr_store_dir = 'data/example.zarr'
baby_zarr_store = zarr.DirectoryStore(baby_zarr_store_dir)
root = zarr.group(baby_zarr_store, overwrite=True)
root.create_dataset('analysed_sst', data=subset, chunks=(2,2,2))

In [ ]:
source_array = root['analysed_sst']
source_array.chunks

In [ ]:
target_chunks = (10,10,10)
max_mem = '1GB'
target_store = 'data/rechunked.zarr'
temp_store = 'data/rechunked-tmp.zarr'
    
array_plan = rechunk(
    source_array,
    target_chunks,
    max_mem,
    target_store,
    temp_store)
array_plan

In [ ]:
future = array_plan.persist()
progress(future)

In [ ]:
a = zarr.open(target_store)
a.chunks

In [ ]:
dsa.from_zarr(target_store)

## A real example

In [10]:
target_store = 'data/analysed_sst/6443x50x100.zarr'
temp_store = 'tmp.zarr'
try:
    shutil.rmtree(target_store)
    shutil.rmtree(temp_store)
except:
    pass
# target_chunks = {
#     'analysed_sst': {'time': 379, 'lat': 439, 'lon': 360},
#     'time': None, # don't rechunk this array
#     'lon': None,
#     'lat': None,
# }
target_chunks = {'time': 6443, 'lat': 50, 'lon': 100}
max_mem = '1GB'

array_plan = rechunk(ds_zarr['analysed_sst'], target_chunks, max_mem, target_store, temp_store)
array_plan

<Rechunked>
* Source      : <zarr.core.Array '/analysed_sst' (6443, 17999, 36000) int16 read-only>
* Intermediate: dask.array<from-zarr, shape=(6443, 17999, 36000), dtype=int16, chunksize=(6443, 50, 1500), chunktype=numpy.ndarray>
* Target      : <zarr.core.Array (6443, 17999, 36000) int16>

In [11]:
with ProgressBar():
    array_plan.execute()

OSError: [Errno 28] No space left on device